<a href="https://colab.research.google.com/github/zhenyiqi/nanoGPT/blob/master/gpt_dev_zhenyiqi_cleaned.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Imports

In [54]:
import torch

In [55]:
from torch import nn

# Prepare dataset

## Download

In [56]:
!wget https://raw.githubusercontent.com/karpathy/char-rnn/master/data/tinyshakespeare/input.txt

--2024-04-12 18:56:15--  https://raw.githubusercontent.com/karpathy/char-rnn/master/data/tinyshakespeare/input.txt
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.111.133, 185.199.109.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1115394 (1.1M) [text/plain]
Saving to: ‘input.txt.1’

input.txt.1         100%[===================>]   1.06M  --.-KB/s    in 0.06s   

2024-04-12 18:56:15 (19.2 MB/s) - ‘input.txt.1’ saved [1115394/1115394]



In [57]:
with open('input.txt', 'r') as f:
  text = f.read()

In [58]:
print(f"length of the dataset is {len(text)}")

length of the dataset is 1115394


## encode characters into numbers (indices)

In [59]:
chars = sorted(list(set(text)))

In [60]:
vocab_size = len(chars)
print(''.join(chars))
print(vocab_size)


 !$&',-.3:;?ABCDEFGHIJKLMNOPQRSTUVWXYZabcdefghijklmnopqrstuvwxyz
65


In [61]:
stoi = {ch:i for i, ch in enumerate(chars)}
itos = {i:ch for i, ch in enumerate(chars)}

In [62]:
encode = lambda s: [stoi[c] for c in s]
decode = lambda i: ''.join([itos[ind] for ind in i])

In [63]:
print(encode("hello there"))
print(decode(encode("hello there")))

[46, 43, 50, 50, 53, 1, 58, 46, 43, 56, 43]
hello there


In [64]:
import torch

In [65]:
data = torch.tensor(encode(text), dtype=torch.long)
print(data.shape, data.dtype)
print(data[:100])

torch.Size([1115394]) torch.int64
tensor([18, 47, 56, 57, 58,  1, 15, 47, 58, 47, 64, 43, 52, 10,  0, 14, 43, 44,
        53, 56, 43,  1, 61, 43,  1, 54, 56, 53, 41, 43, 43, 42,  1, 39, 52, 63,
         1, 44, 59, 56, 58, 46, 43, 56,  6,  1, 46, 43, 39, 56,  1, 51, 43,  1,
        57, 54, 43, 39, 49,  8,  0,  0, 13, 50, 50, 10,  0, 31, 54, 43, 39, 49,
         6,  1, 57, 54, 43, 39, 49,  8,  0,  0, 18, 47, 56, 57, 58,  1, 15, 47,
        58, 47, 64, 43, 52, 10,  0, 37, 53, 59])


## Create train/validation sets

In [66]:
n = int(0.9 * len(data))
train_data = data[:n]
val_data = data[n:]

In [67]:
# notation
block_size = 8 # this is the maximum length of the chunk (sequence length, in other places, this is also donated as T)
train_data[:block_size + 1]

tensor([18, 47, 56, 57, 58,  1, 15, 47, 58])

In [68]:
x = train_data[:block_size] # [18, 47, 56, 57, 58,  1, 15, 47]
y = train_data[1:block_size + 1] # [47, 56, 57, 58,  1, 15, 47, 58]
for t in range(block_size):
  context = x[:t + 1]
  target = y[t]
  print(f"when input is {context}, the target is : {target}")


when input is tensor([18]), the target is : 47
when input is tensor([18, 47]), the target is : 56
when input is tensor([18, 47, 56]), the target is : 57
when input is tensor([18, 47, 56, 57]), the target is : 58
when input is tensor([18, 47, 56, 57, 58]), the target is : 1
when input is tensor([18, 47, 56, 57, 58,  1]), the target is : 15
when input is tensor([18, 47, 56, 57, 58,  1, 15]), the target is : 47
when input is tensor([18, 47, 56, 57, 58,  1, 15, 47]), the target is : 58


In [69]:
# generalize with extra batch size dimension
torch.manual_seed(1337)
batch_size = 4
block_size = 8
def get_batch(split):
  data = train_data if split == 'train' else val_data
  # a random vector of length batch_size, with its value sampled from
  # the range (0, len(data) - block_size), which is all possible values for the
  # the start of a sampled sequence.
  ix = torch.randint(len(data) - block_size, (batch_size, ))
  # now get the samples out, stacking them together
  x = torch.stack([data[i:i+block_size] for i in ix])  # (block_size, batch_size)
  y = torch.stack([data[i+1:i+block_size+1] for i in ix]) # (block_size, batch_size)
  return x, y

In [70]:
xb, yb = get_batch('train')

In [71]:
print('inputs:')
print(xb.shape)
print(xb)
print('targets:')
print(yb.shape)
print(yb)
print('----')

inputs:
torch.Size([4, 8])
tensor([[24, 43, 58,  5, 57,  1, 46, 43],
        [44, 53, 56,  1, 58, 46, 39, 58],
        [52, 58,  1, 58, 46, 39, 58,  1],
        [25, 17, 27, 10,  0, 21,  1, 54]])
targets:
torch.Size([4, 8])
tensor([[43, 58,  5, 57,  1, 46, 43, 39],
        [53, 56,  1, 58, 46, 39, 58,  1],
        [58,  1, 58, 46, 39, 58,  1, 46],
        [17, 27, 10,  0, 21,  1, 54, 39]])
----


In [72]:
for b in range(batch_size):
  for t in range(block_size):
    context = xb[b, :t+1]
    target = yb[b, t]
    print(f"when input is {context.tolist()}, the target is {target}")

when input is [24], the target is 43
when input is [24, 43], the target is 58
when input is [24, 43, 58], the target is 5
when input is [24, 43, 58, 5], the target is 57
when input is [24, 43, 58, 5, 57], the target is 1
when input is [24, 43, 58, 5, 57, 1], the target is 46
when input is [24, 43, 58, 5, 57, 1, 46], the target is 43
when input is [24, 43, 58, 5, 57, 1, 46, 43], the target is 39
when input is [44], the target is 53
when input is [44, 53], the target is 56
when input is [44, 53, 56], the target is 1
when input is [44, 53, 56, 1], the target is 58
when input is [44, 53, 56, 1, 58], the target is 46
when input is [44, 53, 56, 1, 58, 46], the target is 39
when input is [44, 53, 56, 1, 58, 46, 39], the target is 58
when input is [44, 53, 56, 1, 58, 46, 39, 58], the target is 1
when input is [52], the target is 58
when input is [52, 58], the target is 1
when input is [52, 58, 1], the target is 58
when input is [52, 58, 1, 58], the target is 46
when input is [52, 58, 1, 58, 46

In [73]:
xb

tensor([[24, 43, 58,  5, 57,  1, 46, 43],
        [44, 53, 56,  1, 58, 46, 39, 58],
        [52, 58,  1, 58, 46, 39, 58,  1],
        [25, 17, 27, 10,  0, 21,  1, 54]])

# Modeling

### LayerNorm

In [74]:
dropout = 0.2

In [75]:
# class LayerNorm(nn.Module):
#   def __init__(self, dim, eps=1e-5):
#     self.eps = eps
      # gamma and beta are trainable variables
#     self.gamma = torch.ones(dim)
#     self.beta = torch.ones(dim)

#   def forward(self, x):
#     xmean = x.mean(1, keepdim=True)
#     xvar = x.var(1, keepdim=True)
#     xhat = (x - xmean) / torch.sqrt(xvar + self.eps)
#     out = self.gamma * xhat + self.beta
#     return out

#   def parameters(self):
#     return [self.gamma, self.beta]

### self-attention head with dropout

causal self-attention.

1. Originally Q and K are of dimension (T, head_size), ignoring the batch dimension, where `T` is the (max) sequence length (or block size).

1. The overlap between the Q and K is calculate by taking inner product of the two matrix, resulting in a matrix of shape (T, T)
```
     [q1k1  q1k2, q1k3, q1k4, ..., q1kt]
     [q2k1, q2k2, q2k3, q2k4, ..., q2kt]
     [q3k1, q3k2, q3k3, q3k4, ..., q3kt]
     [...                          ....]
     [qtk1, qtk2, ...              qtkt]
```

1. The matrix is masked such that the all $q_i k_j$, where $j > i$ terms should be ignored. We first make them `-∞`.
```
     [q1k1  -inf, -inf, -inf, ..., -inf]
     [q2k1, q2k2, -inf, -inf, ..., -inf]
     [q3k1, q3k2, q3k3, -inf, ..., -inf]
     [...                          ....]
     [qtk1, qtk2, ...              qtkt]
```
After softmax, these `-∞` terms will become zero
1. Then we calculate the weighted value by multiplying this weight (Q * K) with V, which is of dimension (T, head_size).

* input shape: (T, n_embed)
* output shape: (T, head_size)
* what's the input: a sequence of tokens, that acts as Q, K and V; T is sequence length and n_embed is the embedded token.

In [115]:
class SelfAttentionHead(nn.Module):
  def __init__(self, head_size):
    """self-attention head.

    there will be multiple heads. head_size * n_head = n_embed.
    """
    super().__init__()
    self.key = nn.Linear(n_embed, head_size, bias=False)
    self.query = nn.Linear(n_embed, head_size, bias=False)
    self.value = nn.Linear(n_embed, head_size, bias=False)
    # tril is not the parameter of the module, so we call it a buffer
    # (not a parameter)
    self.register_buffer('tril', torch.tril(torch.ones(block_size, block_size)))
    self.dropout = nn.Dropout(dropout)

  def forward(self, x):
    B, T, C = x.shape # C is the head_size here
    k = self.key(x) # (B, T, head_size)
    q = self.query(x) # (B, T, head_size)

    wei = q @ k.transpose(-2, -1) * C ** -0.5 # (B, T, T)
    # masking terms like q_i * k_j, where j > i.
    # The masked matrix looks like

    # [q1k1  -inf, -inf, -inf, ..., -inf]
    # [q2k1, q2k2, -inf, -inf, ..., -inf]
    # [q3k1, q3k2, q3k3, -inf, ..., -inf]
    # [...                          -inf]
    # [qtk1, qtk2, ...              qtkt]

    wei = wei.masked_fill(self.tril[:T, :T] == 0, float('-inf'))

    # The softmax will get normalize all the non-zero terms and make all the
    # -inf terms zero.
    wei = F.softmax(wei, dim=-1) # (B, T, T)

    wei = self.dropout(wei)

    v = self.value(x) # (B, T, head_size)
    out = wei @ v # (B, T, head_size)
    return out

**Question: How many training parameters here?**

The `nn.Linear` will have n_embed * head_size. There are three of them, so we have

**n_embed * head_size * 3** training parameters

### Multi-head attention with a projection

Note that after the SelfAttention head, we have output of shape (T, head_size), where `T` is the (max) sequence length and head_size * num_head = n_embed.

We need to repeat the same thing num_head times. Of course we can trivially do this, resulting in num_head of (T, head_size) matrix. But these matrix will need to be concatenated so that we're back at (T, head_size * num_head) matrix.

Then we do a projection from n_embed to n_embed.

The concatenation is done by `torch.cat()`.

* Input shape: (T, head_size)
* Output shape: (T, n_embed)

In [116]:
class MultiHeadAttention(nn.Module):
  def __init__(self, num_heads, head_size):
    super().__init__()
    self.heads = nn.ModuleList([SelfAttentionHead(head_size) for _ in range(num_heads)])
    self.proj = nn.Linear(n_embed, n_embed, bias=False)
    self.dropout = nn.Dropout(dropout)

  def forward(self, x):
    attention_out = torch.cat([h(x) for h in self.heads], dim=-1)
    out = self.proj(attention_out)
    return out

Question: how many training parameters here?

each SelfAttentionHead has n_embed * head_size * 3 parameters, and we have num_head of heads, so the total number of parameters for heads are n_embed * head_size * num_head * 3 = 3 * n_embed^2, because head_size * num_head == n_embed.

There is another projection layer that contributes to n_embed * n_embed number of paramters. So the total number of parameters in MultiHeadAttention (which includes selfAttentionHead) is

**4 * n_embed ^ 2**

### FeedForward with dropout

Here we do non-fancy stuffs. Feedforward neural network. Within the network, we make things of higher dimension, so the neural network looks like
```
    ||
    ||
    ||
  ||||||
in||||||out
  ||||||
    ||
    ||
    ||
```

* input shape: (T, n_embed)
* output shape: (T, n_embed)

In [118]:
class FeedForward(nn.Module):
  def __init__(self, n_embed):
    super().__init__()
    self.net = nn.Sequential(
        nn.Linear(n_embed, n_embed * 4, bias=False),
        nn.ReLU(),
        nn.Linear(n_embed * 4, n_embed, bias=False),
        nn.Dropout(dropout)
    )

  def forward(self, x):
    return self.net(x)

**Q: how many training parameters here?**

Linear will have (n_embed * n_embed * 4) parameters

ReLu have 0 training parameters.

The second Linear will have (4 * n_embed * n_embed) parameters

So in total, there are

**8 * e_embed^2** parameters in the feedforward component

### Block of (self-attention and feed-forward)

It's basically just (self-attention and feed-forward but we also add layer norms and skip add)

The skip add is applied to each MultiHeadAttention layer and the feedforward layer.

* input shape: (T, n_embed)
* output shape: (T, n_embed)

In [119]:
class Block(nn.Module):
  def __init__(self, n_embed, n_head):
    super().__init__()
    head_size = n_embed // n_head
    self.sa = MultiHeadAttention(n_head, head_size=head_size)
    self.ffwd = FeedForward(n_embed)
    self.ln1 = nn.LayerNorm(n_embed, bias=False)
    self.ln2 = nn.LayerNorm(n_embed, bias=False)

  def forward(self, x):
    x = x + self.sa(self.ln1(x))
    x = x + self.ffwd(self.ln2(x))
    return x

**How many training parameters**.

We add the training parameters of the multi-headed self attention with the feedforward NN, it would be our answer:

12 * n_embed^2

Additionally, we have two LayerNorm layers, each giving n_embed training parameters (bias = false)

2 * n_embed.

In total, it'd be

**2*n_embed + 12 * n_embed^2**

### Model with multiple layers of (self-attention, feed-forward) and layernorm + skipadd

Put everything together and repeat the decoder module n_layer times with skipAdd layers

* input shape: (T)
* output shape: (T, vocab_size) + a scaler (the loss)

In [121]:
import torch
import torch.nn as nn
from torch.nn import functional as F
torch.manual_seed(1337)

n_embed = 32
n_head = 4
n_layer = 3

class GPT(nn.Module):
  def __init__(self):
    super().__init__()
    # an embedding is a table that maps each token to a vector
    # nn.Embedding(n1, n2) maps a token whose value is up to n1 (starting from 0)
    # to a vector of size n2. Here we map each token to a vector of size vocab_size
    # (One example is one-hot encoding)
    self.token_embedding_table = nn.Embedding(vocab_size, n_embed)
    self.position_embedding_table = nn.Embedding(block_size, n_embed)
    self.blocks = nn.Sequential(*[Block(n_embed, n_head) for _ in range(n_layer)])
    self.ln_f = nn.LayerNorm(n_embed, bias=False) # final layer norm
    # This is the output layer that maps the transformed embeddings back to the
    # original vocab_size vector - logits.
    self.lm_head = nn.Linear(n_embed, vocab_size, bias=False)

  def forward(self, idx, targets=None):
    """A Forward proporgation.

    idx: the input. Dimension [B, T].
    targets: target. Dimension [B, T], [B, i]th element is the target of sequence
      idx[B, :i].
    """
    B, T = idx.shape

    # idx and targets are both (B, T), meaning (batch_size, block_size)
    token_embed = self.token_embedding_table(idx) # (B, T, C = n_embed)
    # pos_embed = self.position_embedding_table(torch.arange(T, device=device)) # (T, C)
    pos_embed = self.position_embedding_table(torch.arange(T)) # (T, C)

    x = token_embed + pos_embed
    x = self.blocks(x)
    x = self.ln_f(x)
    logits = self.lm_head(x) # (B, T, C = vocab_size)
    # here is a bit tricky, please refer to
    # https://pytorch.org/docs/stable/generated/torch.nn.functional.cross_entropy.html
    # to understand what's in there
    if targets is None:
      loss = None
    else:
      B, T, C = logits.shape
      # reshapes the logits to a 2D tensor where each row represents a single
      # data sample and each column represents a class
      logits = logits.view(B * T, C)
      # reshapes the targets to a 1D tensor where each element represents the
      # true class label for a single data sample
      targets = targets.view(B * T)

      loss = F.cross_entropy(logits, targets)

    return logits, loss

  def generate(self, idx, max_new_tokens):
    """Generate the next token for max_new_tokens of times for the batch idx.

    idx: a batch of samples, of size (B, T).
    max_new_tokens: int, how many tokens to be generated given a sample.
    """
    for _ in range(max_new_tokens):
      # crop the input size so that it's at most the same as block_size when
      # generating
      # for all the batches (idx[:, ...]), we choose the last block_size number
      # of tokens; dimension (B, T)
      idx_cond = idx[:, -block_size:]
      # get the predictions
      logits, loss = self(idx_cond) # logits: (B, T, C)
      # Last one of the sequence
      logits = logits[:, -1, :] # (B, C)
      probs = F.softmax(logits, dim=-1) # (B, C)
      idx_next = torch.multinomial(probs, num_samples=1) # (B, 1)
      idx = torch.cat((idx, idx_next), dim=1) # (B, T + 1)
    return idx

model = GPT()
logits, loss = model(xb, yb)
print(logits.shape)
print(loss)

torch.Size([256, 65])
tensor(4.2418, grad_fn=<NllLossBackward0>)


**What's the number of training parameters?**

* nn.Embedding for the tokens themselves has a dimension of (vocab_size, n_embed) so it contains (vocab_size * n_embed) parameters
* nn.Embedding for the positions contains (T * n_embed)
* now we have n_layer layers of decoder, each decoder, as we found earlier, has (12 * n_embed^2 + 2 * n_embed) number of parameters so in total we have 12 * n_layer * n_embed^2 + 2 * n_layer * n_embed parameters
* we also add another LayerNorm layer at the end, contributing to n_embed number of parameters.
* lastly, we have a linear head that maps the vector of length n_embed to vector of length vocab_size, this will give another (vocab_size * n_embed) parameters. adding them up, we got

**(2 vocab_size + T + 2 n_layer + 1)  n_embed + 12 n_layer n_embed ^ 2** number of parameters

For this one.
n_embed = 32, n_layer = 3, vocab_size = 65
T = 8 (determined dynamically during training and reused during inference)

145 * 32 + 36 * 32 ^ 2 = 41504

In [122]:
sum(p.numel() for p in model.parameters())

41504

# Training

In [82]:
# Take model parameters as the input. This means that an optimizer is the object
# that manages and iterates the model weights.
optimizer = torch.optim.AdamW(model.parameters(), lr=1e-3)

## Constants

In [83]:
max_iters = 5000
eval_interval = 500
learning_rate = 1e-3
device = 'cuda' if torch.cuda.is_available() else 'cpu'

eval_iters = 200

In [84]:
# For evaluation only
# To tell Pytorch that everything inside this function will not be used in
# loss.backward() so that it won't allocate any memory for the local variables
# for backproporgation.
@torch.no_grad()
def estimate_loss():
  out = {}
  model.eval()
  for split in ['train', 'val']:
    losses = torch.zeros(eval_iters)
    for k in range(eval_iters):
      X, Y = get_batch(split)
      logits, loss = model(X, Y)
      losses[k] = loss.item()
    out[split] = losses.mean()
  model.train()
  return out

In [85]:
batch_size = 32
for iter in range(max_iters):
  # estimate the loss every 500 iterations
  if iter % eval_interval == 0:
    losses = estimate_loss()
    print(f"step {iter}: train loss {losses['train']:.4f}, val loss {losses['val']:.4f}")

  # sample a batch of data
  xb, yb = get_batch('train')

  # evaluate the loss
  logits, loss = model(xb, yb)
  # zero out the gradients before the training
  optimizer.zero_grad(set_to_none=True)
  loss.backward()
  optimizer.step()

step 0: train loss 4.3104, val loss 4.3100
step 500: train loss 2.4657, val loss 2.4560
step 1000: train loss 2.3115, val loss 2.3273
step 1500: train loss 2.2429, val loss 2.2598
step 2000: train loss 2.1853, val loss 2.2123
step 2500: train loss 2.1563, val loss 2.1893
step 3000: train loss 2.1411, val loss 2.1585
step 3500: train loss 2.1024, val loss 2.1458
step 4000: train loss 2.0894, val loss 2.1271
step 4500: train loss 2.0581, val loss 2.0932


# Eval

In [86]:
idx = torch.zeros((1, 1), dtype=torch.long)

print(decode(model.generate(idx, max_new_tokens=300)[0].tolist()))


MEONTUCES:
Leash for hous?
Tais magelianss that, tot shee perecaint:
A nestiones'd me; but etsed a thoGough
Cetrane:
Aod theag
Furs'd sownerreawirs that she aland letgieg,
And yough, guk thatance opinair! vive; the loot.

I wethyit? pare,
Thirphe haty of not oun, cony the him wo af that's of mitte t


# Save the checkpoint

In [87]:
! ls

input.txt  input.txt.1	sample_data


In [88]:
output_dir = "./checkpoints"

In [91]:
checkpoint = {
    'epoch': 5000,
    'model_state_dict': model.state_dict(),
    'optimizer_state_dict': optimizer.state_dict(),
}

In [92]:
checkpoint_filename = 'model_checkpoint.pt'
checkpoint_path = 'checkpoints/'

In [94]:
import os

In [96]:
torch.save(checkpoint, os.path.join(checkpoint_path, checkpoint_filename))

# Load from a checkpoint

In [97]:
another_model = GPT()


In [98]:
checkpoint_path = 'checkpoints/model_checkpoint.pt'

In [99]:
checkpoint = torch.load(checkpoint_path)

In [101]:
another_model.load_state_dict(checkpoint['model_state_dict'])
another_optimizer = torch.optim.AdamW(another_model.parameters(), lr=1e-3)

In [102]:
another_optimizer.load_state_dict(checkpoint['optimizer_state_dict'])

In [104]:
idx = torch.zeros((1, 1), dtype=torch.long)

print(decode(another_model.generate(idx, max_new_tokens=300)[0].tolist()))


CRICPARoEl:'sels treatson
Thany Ourth's a frind allor foull feralt'seforthe hath stink: bagiyily.

KINGWCARBYEENAPER:
LUEINGTABENRGCASIR:
Buth-
Whancch thou wenthed a gringinss of way you not that thene the hould; thut on posentay Kords
Our lines; in not
andyn I?
Yevince with sold,
So shairse, in th


# Upload my model to HuggingFace (the code doesn't work yet)


In [105]:
from huggingface_hub import HfApi

In [106]:
from google.colab import userdata
hf_token = userdata.get('HuggingFaceToken')

In [109]:
api = HfApi(token=hf_token)

In [110]:
model_name = "mini-gpt"